In [1]:
from start import *
df = pd.read_csv('df_cleaned.csv', decimal='.')
cols_text=['review_comment_title', 'review_comment_message']
cols_cat=['product_category_name','review_score']
cols_num=['price','freight_value', 'order_item_id']
df_copy = df.copy()

C:\Users\boliv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Gera o embedding para as duas colunas textuais
elmo_result_message = gera_embbeding(data=df,modelo='elmo', coluna='review_comment_message')
elmo_result_title = gera_embbeding(data=df,modelo='elmo', coluna='review_comment_title')

2024-08-28 23:45:58,746 WARNING: Could not find config.  Trying elmoformanylangs/168/cnn_50_100_512_4096_sample.json
2024-08-28 23:45:58,814 INFO: char embedding size: 2237
2024-08-28 23:45:59,516 INFO: word embedding size: 178961
2024-08-28 23:46:05,908 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(178961, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2237, 50, padding_idx=2234)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleL

In [15]:
#Gera as colunas PCA que serão utilizadas no Ahmad, para o artigo esse código foi rodado 3 vezes, com o pca_k = 3, pca_k = 6, pca_k = 12
df_elmo_pca_message = criando_coluna_PCA(df=df, modelo='elmo', col='message', result_embbe=elmo_result_message, pca_k=12)
df_elmo_pca_title = criando_coluna_PCA(df=df_copy, modelo='elmo', col='title', result_embbe=elmo_result_title, pca_k=12)

In [17]:
#Seleciona apenas as variavéis numéricas
numeric_data_message = df_elmo_pca_message.select_dtypes(include=['float64', 'float32'])
numeric_data_title = df_elmo_pca_title.select_dtypes(include=['float64', 'float32'])
df_mixed = pd.concat([numeric_data_message, numeric_data_title], axis=1)
df_mixed = df_mixed.loc[:, ~df_mixed.columns.duplicated()]

In [18]:
#Gera a distância Ahmad para os dados mistos
dist_mtx_mt=distmix(data=df_mixed, method="ahmad", idnum=df_mixed.columns, idcat=cols_cat)

In [55]:
#Foram rodados experimentos alterando o n_clusters e o linkage, como mostrado no artigo
clustering_embedding = AgglomerativeClustering(metric='precomputed', linkage='single', n_clusters=6).fit_predict(dist_mtx_mt)

#APLICANDO SILHUETA EMBEDDIING
print(silhouette_score(X = dist_mtx_mt, labels=clustering_embedding, metric='precomputed'))

0.9286532969811249
